## Import Libraries

In [43]:
# Common imports
import os
import numpy as np

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [44]:
# TensorFlow imports
# may differs from version to versions

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing import image

## Splitting images

In [45]:
# import splitfolders  # or import split_folders

# Split dengan ratio.
# Untuk hanya membagi menjadi set pelatihan dan validasi, setel tuple menjadi `ratio`, i.e, `(.8, .2)`.
# splitfolders.ratio("dataset", output="face_dataset_test_images", seed=1337, ratio=(.8,.2), group_prefix=None)


# Dataset information

# Test dataset is set explicitly, because the amount of data is very small
train_image_folder = os.path.join('datasets', 'face_dataset_train_aug_images')
test_image_folder = os.path.join('datasets', 'face_dataset_test_images')
img_height, img_width = 250, 250  # size of images
num_classes = 3 

# Training settings
validation_ratio = 0.15  # 15% for the validation
batch_size = 16

AUTOTUNE = tf.data.AUTOTUNE

In [46]:
# Train and validation sets
train_ds = keras.preprocessing.image_dataset_from_directory(
    train_image_folder,
    validation_split=validation_ratio,
    subset="training",
    seed=42,
    image_size=(img_height, img_width),
    label_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

val_ds = kera=s.preprocessing.image_dataset_from_directory(
    train_image_folder,
    validation_split=validation_ratio,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=True)

Found 1200 files belonging to 3 classes.
Using 1020 files for training.
Found 1200 files belonging to 3 classes.
Using 180 files for validation.


In [47]:
# Test set
test_ds = keras.preprocessing.image_dataset_from_directory(
    test_image_folder,
    image_size=(img_height, img_width),
    label_mode='categorical',
    shuffle=False)

Found 60 files belonging to 3 classes.


In [48]:
class_names = test_ds.class_names
class_names

['abizar', 'bintang', 'muchdor']

## Modelling with ResNet50

In [49]:
base_model = keras.applications.ResNet50(weights='imagenet',
                                         include_top=False,  # without dense part of the network
                                         input_shape=(img_height, img_width, 3))

# Set layers to non-trainable
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of ResNet
global_avg_pooling = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(num_classes, activation='sigmoid')(global_avg_pooling)

face_classifier = keras.models.Model(inputs=base_model.input,
                                     outputs=output,
                                     name='ResNet50')
face_classifier.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 250, 250, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 256, 256, 3)  0           ['input_9[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 125, 125, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [50]:
# ModelCheckpoint to save model in case of interrupting the learning process
checkpoint = ModelCheckpoint("models/face_classifier.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only=True,
                             verbose=1)

# EarlyStopping to find best model with a large number of epochs
earlystop = EarlyStopping(monitor='val_loss',
                          restore_best_weights=True,
                          patience=3,  # number of epochs with no improvement after which training will be stopped
                          verbose=1)

callbacks = [earlystop, checkpoint]

In [51]:
face_classifier.compile(loss='categorical_crossentropy',
                        optimizer=keras.optimizers.Adam(learning_rate=0.01),
                        metrics=['accuracy'])

## Training

In [55]:
epochs = 5

history = face_classifier.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds)

face_classifier.save("models/face_classifier.h5")

Epoch 1/5
64/64 [==============================] - ETA: 0s - loss: 8.5272e-06 - accuracy: 1.0000
Epoch 00001: val_loss improved from 0.00000 to 0.00000, saving model to models\face_classifier.h5
64/64 [==============================] - 191s 3s/step - loss: 8.5272e-06 - accuracy: 1.0000 - val_loss: 1.3120e-06 - val_accuracy: 1.0000
Epoch 2/5
64/64 [==============================] - ETA: 0s - loss: 5.8929e-06 - accuracy: 1.0000
Epoch 00002: val_loss improved from 0.00000 to 0.00000, saving model to models\face_classifier.h5
64/64 [==============================] - 174s 3s/step - loss: 5.8929e-06 - accuracy: 1.0000 - val_loss: 9.8546e-07 - val_accuracy: 1.0000
Epoch 3/5
64/64 [==============================] - ETA: 0s - loss: 4.4056e-06 - accuracy: 1.0000
Epoch 00003: val_loss improved from 0.00000 to 0.00000, saving model to models\face_classifier.h5
64/64 [==============================] - 177s 3s/step - loss: 4.4056e-06 - accuracy: 1.0000 - val_loss: 7.4174e-07 - val_accuracy: 1.0000
E